In [9]:
# import functions
import decoherence_mapping_functions as dmf
from decoherence_mapping_parameters import nuclear, electronic

In [10]:
B0 = 5.1*1e-3
theta_arr = dmf.np.linspace(0,dmf.np.pi*(3/4),68)
phi_arr = dmf.np.linspace(0,2*dmf.np.pi,181)
theta,phi = dmf.np.meshgrid(theta_arr,phi_arr,indexing='ij')
theta = theta.flatten()
phi = phi.flatten()
Bx = B0*dmf.np.sin(theta)*dmf.np.cos(phi)
By = B0*dmf.np.sin(theta)*dmf.np.sin(phi)
Bz = B0*dmf.np.cos(theta)
# creating field array of size 3*len(Bz)
B_arr = dmf.np.stack((Bx,By,Bz),axis=0)
# neighbouring fields of (0,0,0)/ eighteen bias fields (18*3)
arr = dmf.np.array([
    # Axial terms (6)
    [0.1, 0, 0], [-0.1, 0, 0],
    [0, 0.1, 0], [0, -0.1, 0],
    [0, 0, 0.1], [0, 0, -0.1],
    # Cross terms (12)
    [0.1, 0.1, 0], [-0.1, 0.1, 0], [0.1, -0.1, 0], [-0.1, -0.1, 0],  # XY-plane
    [0.1, 0, 0.1], [-0.1, 0, 0.1], [0.1, 0, -0.1], [-0.1, 0, -0.1],  # XZ-plane
    [0, 0.1, 0.1], [0, -0.1, 0.1], [0, 0.1, -0.1], [0, -0.1, -0.1]   # YZ-plane
]) * 1e-3
# creating 18*3 array to 18*3*len(Bz) array/ eighteen bias fields around each magnetic field (18*3*len(Bz))
result_arr = dmf.np.tile(arr[:, :, dmf.np.newaxis], (1, 1, B_arr.shape[1]))
# all the points where we will solve hamiltonian
neighbour_arr = dmf.np.stack([B_arr] * 18, axis=0) + result_arr

In [11]:
# NVB (Strongly coupled C13 present)
data = dmf.curvature_transition_energy([electronic('VB-'),[nuclear('14N11'),nuclear('14N12'),nuclear('14N13')]], B_arr, neighbour_arr, dmf.generalized_operators, dmf.generalized_hamiltonian)

  0%|          | 0/12308 [00:00<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/basantamistri/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/9w/zrdh5v0d5vzgt44486kqx7s80000gn/T/ipykernel_82511/2154093309.py", line 2, in <module>
    data = dmf.curvature_transition_energy([electronic('VB-'),[nuclear('14N11'),nuclear('14N12'),nuclear('14N13')]], B_arr, neighbour_arr, dmf.generalized_operators, dmf.generalized_hamiltonian)
  File "/Users/basantamistri/Downloads/dfs/github/decoherence_mapping_functions.py", line 134, in curvature_transition_energy
IndexError: tuple index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/basantamistri/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/basant

In [ ]:
# Spherical representation of Average(over all possible combinations of two manifolds) 
# Transition Energy, Gradient, and Curvature

# Generate theta, phi grid
theta, phi = dmf.np.meshgrid(
    dmf.np.linspace(0, dmf.np.pi * (3 / 4), 68),
    dmf.np.linspace(0, 2 * dmf.np.pi, 181),
    indexing='ij'
)
theta_flat = theta.flatten()
phi_flat = phi.flatten()

# Magnetic field components
B_0 = 5.1 * 1e-3
B_x = B_0 * dmf.np.sin(theta_flat) * dmf.np.cos(phi_flat)
B_y = B_0 * dmf.np.sin(theta_flat) * dmf.np.sin(phi_flat)
B_z = B_0 * dmf.np.cos(theta_flat)

# Create subplot figure: 1 row, 3 columns
fig = dmf.make_subplots(
    rows=1, cols=3,
    specs=[[{'type': 'surface'}]*3],
    subplot_titles=["Energy", "Gradient", "Curvature"]
)

# Magnetic field coordinates
B_coords = dict(x=B_x, y=B_y, z=B_z)

# Use only 3 subplots (Energy, Gradient, Curvature)
coloraxis_names = ['coloraxis1', 'coloraxis2', 'coloraxis3']

# Define colorbar positions for each subplot
colorbar_positions = [
    dict(x=0.30, y=0.55),  # Left
    dict(x=0.66, y=0.55),  # Center
    dict(x=1.00, y=0.55)   # Right
]

# Only use indices 0, 2, 3 corresponding to Energy, Gradient, Curvature
data_indices = [0, 2, 3]

# Add traces for Energy, Gradient, Curvature
# Add traces
for i in range(3):
    row, col = divmod(i, 2)
    row += 1
    col += 1

    intensity = []
    for j in range (len(B_x)):
        intensity.append((data[i][j, 19, 4]))

    mesh = dmf.go.Mesh3d(
        x=B_coords['x'], y=B_coords['y'], z=B_coords['z'],
        intensity=dmf.np.array(intensity),
        colorscale='viridis',
        coloraxis=coloraxis_names[i],
        opacity=1.0,
        alphahull=0
    )

    fig.add_trace(mesh, row=row, col=col)

    # Add color axis layout
    fig.update_layout({
        coloraxis_names[i]: dict(
            colorscale='viridis',
            showscale=True,
            colorbar=dict(
                x=colorbar_positions[i]['x'],
                y=colorbar_positions[i]['y'],
                len=0.6,
                thickness=10
            )
        )
    })

# Final layout tweaks
fig.update_layout(
    template='plotly_white',
    height=500,
    width=1200,
    title='Average Transition Energy, Gradient, and Curvature @ 5.1mT VB⁻ in hBN',
    scene=dict(bgcolor='rgba(0,0,0,0)'),
    scene2=dict(bgcolor='rgba(0,0,0,0)'),
    scene3=dict(bgcolor='rgba(0,0,0,0)')
)

fig.show()
